# **국민 연금 블로그 크롤링**

In [1]:
from tqdm import trange, notebook

In [2]:
# 라이브러리 불러오기
from selenium import webdriver # 브라우저 자동화를 위한 모듈
import requests # HTTP 요청을 보내기 위한 모듈
from bs4 import BeautifulSoup as BS # HTML 내용 파싱을 위한 모듈
import pandas as pd # 데이터 조작 및 분석을 모듈
import time # 코드 실행 속도 조절을 위한 모듈
import re # 정규 표현식 사용을 위한 모듈
import pickle # 파이썬 객체 직렬화를 위한 모듈
from selenium.webdriver.common.by import By # 다양한 방법으로 엘리먼트를 찾기 위한 모듈
import datetime # 날짜와 시간 연산을 위한 모듈
import time
import numpy as np
import pandas as pd
from threading import Thread
from datetime import datetime
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

## **크롤링 속도 개선을 위한 멀티쓰레딩 개념을 이용한 크롤링 방법으로 진행**

* 멀티쓰레딩을 사용하지 않으면 한 개의 크롬창만 띄워서 크롤링을 진행하는 탓에 속도가 매우 느립니다. 하지만 멀티쓰레딩 개념을 이용하면 cpu의 다중 코어에 의해 여러개의 크롬을 병렬적으로 띄워 작업이 가능하게 됩니다.
* 예전의 cpu는 single core 혹은 dual core 였지만 현대의 cpu는 하드웨어에 대한 비약적인 발전으로 인해 multi core의 형태를 띄고 있습니다.
* 쓰레드는 한 프로세스 안에서 데이터를 공유하기 때문에 뉴스 제목, 뉴스기사, 뉴스 발행 날짜가 저장 될 수 있는 공유 리스트를 생성해서 각 쓰레드들이 생성하는 데이터를 실시간으로 합치는 방법을 채택 했었으나 이러한 경우에 여러개의 쓰레드가 뉴스제목, 뉴스기사, 뉴스 발행 날짜를 병렬적으로 append 할 경우 뉴스 제목,뉴스 기사, 뉴스 발행 날짜의 순서들이 꼬일 우려가 있기에 이를 각각의 쓰레드에서 지역 데이터 영역에서 리스트를 생성 후 쓰레드의 작업이 모두 끝나면 프로세스 공유 리스트에 한번에 추가하는 방법을 채택 하였습니다. 

In [4]:
# 병렬로 수행될 크롤링 함수
def crawl(keyword, start_date, end_date):

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-extensions")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    global globa_publish_date, global_new_doc, global_like_cnt, global_comment_cnt, global_comment_list, global_img_cnt, global_div_cnt,global_title_list, global_error_url_list
    
    url = f"https://search.naver.com/search.naver?ssc=tab.blog.all&query={keyword}&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom{start_date}to{end_date}"
    driver.get(url)
    endScrollDown(driver)
    titles = driver.find_elements(By.CLASS_NAME, 'title_link')
    
    url_list = []  # URL을 저장할 리스트
    local_new_doc=[]
    local_like_cnt=[]
    local_comment_cnt=[]
    local_comment_list=[]
    local_img_cnt=[]
    local_div_cnt=[]
    local_title_list= []
    local_error_url_list= []
    local_publish_date= []

    for i, title_element in enumerate(titles):
        try:
            local_title_list.append(title_element.text) # titles 리스트에서 제목을 추출해 title_list에 추가
            local_publish_date.append(start_date)
            url_list.append(title_element.get_attribute('href'))  # titles 리스트에서 URL을 추출해 url_list에 추가
        except:
            print("오류 발생")
            continue  # 오류가 발생해도 다음 항목으로 계속 진행
    
    # 블로그 글 크롤링
    for i in notebook.tqdm(url_list):
        url_path = i # url 불러오기
        driver.switch_to.window(driver.window_handles[0]) # 첫 번째 탭으로 이동
        driver.execute_script("window.open('{}')".format(url_path)) # url 실행
        driver.switch_to.window(driver.window_handles[1]) # 두 번째 탭으로 이동

        # time.sleep(1) # 1초 대기
        try:
            iframes = driver.find_elements(By.TAG_NAME, 'iframe') 
            d = '' # 댓글을 저장할 변수 초기화

              # 댓글영역의 HTML코드 가져오기
            if len(iframes) > 0:   # iframe의 존재 확인
                driver.switch_to.frame(0) # 첫번째 iframe으로 전환하여 해당 iframe의 내용을 가져옴
                html = driver.page_source #  현재 페이지의 HTML 코드를 가져와서 html 변수에 저장
                soup = BS(html, 'html.parser') # html변수에 저장된 HTML코드를 파싱하여 soup을 생성 (뎃글영역에 대한 HTML코드 포함) 

                try:
                    # 블로그 본문 추출 (se-main-container 클래스를 찾아서 본문 텍스트를 가져옴)
                    a = soup.find('div', class_= 'se-main-container').get_text() 

                except:
                    # 블로그 본문을 찾지 못한 경우 (일반 블로그의 경우 id가 postListBody 인 요소를 본문으로 가져옴)
                    a = soup.find('div', id='postListBody') 
                    a = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "", str(a))  # 정규표현식을 이용하여 한글만 남김

                try:
                    # 좋아요 수 추출 (u_cnt _count 클래스를 찾아서 좋아요 수 텍스트를 가져옴)
                    b = soup.find('em', class_='u_cnt _count').get_text()
                except:
                    # 좋아요 수를 찾지 못한 경우
                    b = 'null'

                try:
                    # 댓글 수 추출 (commentCount id를 찾아서 댓글 수 텍스트를 가져옴)
                    c = soup.find('em', id="commentCount").get_text()
                except:
                    # 댓글 수를 찾지 못한 경우
                    c = 'null'

                try:
                    # 댓글을 모두 보기 위해 댓글 버튼 클릭
                    comment = driver.find_elements(By.CLASS_NAME, 'btn_arr')
                    comment[len(comment) - 1].click()  # 마지막 댓글버튼 클릭
                    time.sleep(1)

                    commentLen = len(driver.find_elements(By.CLASS_NAME, 'u_cbox_page'))
                    d = ''
                    # 각 댓글 페이지의 댓글 추출
                    for _ in range(0, commentLen) :
                        commentList = driver.find_elements(By.CLASS_NAME, 'u_cbox_text_wrap')
                        for j in range(len(commentList)):
                            commentList[j] = commentList[j].text
                    soup = BS(html, 'html.parser')

                    # 추출한 댓글들을 줄바꿈 문자로 구분하여 하나의 문자열로 저장
                    d = "\n".join(commentList)

                except:
                    # 댓글을 찾지 못한 경우
                    d = 'null'

                # 이미지 수 추출 (se-image-resource egjs-visible 클래스를 찾아서 이미지 수를 가져옴)
                e = len(soup.find_all('img', class_='se-image-resource egjs-visible'))

                # 영상 수 추출 (pzp-ui-dimmed pzp-dimmed pzp-pc__dimmed 클래스를 찾아서 영상 수를 가져옴)
                f = len(soup.find_all('div', class_='pzp-ui-dimmed pzp-dimmed pzp-pc__dimmed'))

                # 추출한 데이터를 각각의 리스트에 추가
                local_new_doc.append(a)
                local_like_cnt.append(b)
                local_comment_cnt.append(c)
                local_comment_list.append(d)
                local_img_cnt.append(e)
                local_div_cnt.append(f)
                driver.switch_to.default_content() # iframe에서 기본 컨텐츠로 전환

            else:
                local_new_doc.append(' ') # 본문이 없을 경우 빈 문자열 추가
                local_like_cnt.append(' ') # 좋아요 수가 없을 경우 빈 문자열 추가
                local_comment_cnt.append(' ') # 댓글 수가 없을 경우 빈 문자열 추가
                local_comment_list.append(' ') # 댓글이 없을 경우 빈 문자열 추가
                local_img_cnt.append(' ') # 이미지 수가 없을 경우 빈 문자열 추가
                local_div_cnt.append(' ') # 영상 수가 없을 경우 빈 문자열 추가

        except:
            local_new_doc.append(' ') # 예외 발생 시 빈 문자열 추가
            local_like_cnt.append(' ') # 예외 발생 시 빈 문자열 추가
            local_comment_cnt.append(' ') # 예외 발생 시 빈 문자열 추가
            local_comment_list.append(' ') # 예외 발생 시 빈 문자열 추가
            local_img_cnt.append(' ') # 예외 발생 시 빈 문자열 추가
            local_div_cnt.append(' ') # 예외 발생 시 빈 문자열 추가
            continue
            
        driver.close()
#         time.sleep(0.3) # 0.3초 대기


#     for news in notebook.tqdm(url_list):
#         try:
            
#             article= Article(news)
#             article.download()
#             article.parse()
#             local_title_list.append(article.title)
#             local_new_doc.append(article.text)
#             local_publish_date.append(article.publish_date)
            
#         except:
#             local_error_url_list.append(news)
#             continue

    global_title_list.extend(local_title_list)
    global_new_doc.extend(local_new_doc)
    global_like_cnt.extend(local_like_cnt)
    global_comment_cnt.extend(local_comment_cnt)
    global_comment_list.extend(local_comment_list)
    global_img_cnt.extend(local_img_cnt)
    global_div_cnt.extend(local_div_cnt)
    global_error_url_list.extend(local_error_url_list)
    global_publish_date.extend(local_publish_date)

    driver.quit() # 크롬 닫기

# 스크롤 다운 함수에 driver 인자 추가
def endScrollDown(driver):
    end = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        start = driver.execute_script("return document.body.scrollHeight")
        if start == end:
            break
        end = start

# 메인 코드
if __name__ == "__main__": 
    global_title_list=[]
    global_new_doc=[]
    global_like_cnt=[]
    global_comment_cnt=[]
    global_comment_list=[]
    global_img_cnt=[]
    global_div_cnt=[]
    global_error_url_list=[]
    global_publish_date= []
    #global_new_doc = [] # 본문을 저장할 공유 리스트 초기화
    #global_publish_date= [] # 신문 작성 날짜 저장할 공유 리스트 초기화
    #global_title_list = []  # 제목을 저장할 공유 리스트
    #global_error_url_list=[]
    start_time= datetime.now()

    keyword = '국민연금'
    start_date = ['20220501', '20220601', '20220701', '20220801', '20220901', '20221001', '20221101', '20221201', 
              '20230101', '20230201', '20230301', '20230401', '20230501', '20230601', '20230701', '20230801', 
              '20230901', '20231001', '20231101', '20231201', '20240101', '20240201', '20240301', '20240401']

    end_date =   ['20220531', '20220630', '20220731', '20220831', '20220930', '20221031', '20221130', '20221231', 
              '20230131', '20230228', '20230331', '20230430', '20230531', '20230630', '20230731', '20230831', 
              '20230930', '20231031', '20231130', '20231231', '20240131', '20240229', '20240331', '20240430']


    threads = []
    for i in range(len(start_date)):
        thread = Thread(target=crawl, args=(keyword, start_date[i], end_date[i]))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()  # 모든 스레드가 종료될 때까지 대기

    end_time= datetime.now()
    print("걸린시간: ", end_time - start_time)


  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

...

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

걸린시간:  1:33:14.773075


### **12개의 쓰레드를 사용하여 1시간 18분(78분)이 소요됐다. 만약 멀티쓰레딩을 사용하지 않았다면 약 15시간 36분(936분)이 소요 되었을 것이다.**

In [6]:
print("걸린 시간: ", end_time - start_time)

걸린 시간:  1:33:14.773075


In [7]:
print(len(global_title_list))

print(len(global_new_doc)) 

print(len(global_like_cnt))

print(len(global_comment_cnt))

print(len(global_comment_list))

print(len(global_img_cnt))

print(len(global_div_cnt))

print(len(global_publish_date))

24480
24480
24480
24480
24480
24480
24480
24480


In [18]:
# 크롤링한 데이터를 데이터프레임으로 변환
pension_blog = pd.DataFrame() 
pension_blog['title'] = global_title_list
pension_blog['doc'] = global_new_doc
pension_blog['like_cnt'] = global_like_cnt
pension_blog['comment_cnt'] = global_comment_cnt
pension_blog['comment_list'] = global_comment_list
pension_blog['img_cnt'] = global_img_cnt
pension_blog['div_cnt'] = global_div_cnt
pension_blog['publish_date']= global_publish_date
pension_blog['ch'] = 'naver'
pension_blog['ch2'] = 'blog'

In [19]:
pension_blog

,title,doc,like_cnt,comment_cnt,comment_list,img_cnt,div_cnt,publish_date,ch,ch2
0,국세체납 건강보험료 미납시 국민연금 연체 압류 때문에 고민한다면,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n​​해마다 매년 경제가 불...,4,null,null,12,0,20231201,naver,blog
1,국세체납 건강보험료 미납시 국민연금 연체 압류 어떻게 해결할까,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n​​경제가 침체되다 보니 ...,5,null,null,12,0,20231201,naver,blog
2,국세체납 건강보험료 미납 국민연금 연체 압류 등 세금 문제 해결하려면,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n​우리나라 국민은 납세의 ...,8,null,null,11,0,20231201,naver,blog
3,국세체납 건강보험료 미납시 국민연금 연체 압류 등 개인회생 세금 면책 가능할까,\n\n\n\n\n국세체납 건강보험료 미납시 국민연금 연체 압류 등 개인회생 세금 ...,3,null,null,14,0,20231201,naver,blog
4,"개인사업자 사업자등록 건강보험료, 국민연금 납부는 어떻게? 주부 프리랜서의 변신 (1)",\n\n\n\n\n사업자등록하면어떤 일이 벌어지는지 진짜알고 계신건가요?\n\n\n...,71,\n\t\t\t\t\t\t\t\t\t\t\t26\n\t\t\t\t\t\t\t\t\t\t,비밀 댓글입니다.\n지역가입자 전환되면 납부하는 보험료가 상당하다 하더라구요 ㅠㅠ\...,7,0,20231201,naver,blog
...,...,...,...,...,...,...,...,...,...,...
24475,기초노령연금 수급자격 금액 신청방법 주의점,\n\n\n\n\n기초연금 수급자격\n\n\n\n \n\n\n\n\n\n\n\n\n...,151,\n\t\t\t\t\t\t\t\t\t\t\t14\n\t\t\t\t\t\t\t\t\t\t,\n비밀 댓글입니다.\n너무나 좋은정보 이네용\n기초노령연금 수급자격 도움 되시길 ...,6,0,20230501,naver,blog
24476,노인기초연금 기초노령연금 재산 금액 부부 나이 감액 신청에 대해 알려주세요,\n\n\n\n\n노인기초연금 재산 금액 기초노령연금 나이 부부 신청\n\n\n\n...,,\n\t\t\t\t\t\t\t\t\t\t\t10\n\t\t\t\t\t\t\t\t\t\t,정성스런 포스팅 잘보고 갑니다\n좋은하루 되세요\n자세한 연금소식 감사합니다. 어르...,6,0,20230501,naver,blog
24477,국민연금도 투자하는 부동산,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n올해 5월 초에 국민연금이...,,\n,,2,0,20230501,naver,blog
24478,2023년 기초노령연금 신청 방법 수급자격 및 금액,\n\n\n\n\n요즘 들어 연금에 대한 포스팅이 부쩍이나 늘어난 것 같습니다. 연...,,\n\t\t\t\t\t\t\t\t\t\t\t10\n\t\t\t\t\t\t\t\t\t\t,오늘두 즐건 하루보내세요\n유용한 포스팅 잘 보고 갑니다.^^ 감사합니다.\n들렸다...,7,0,20230501,naver,blog


In [ ]:
# 데이터프레임을 pickle파일로 저장
f = open("../data/raw_data/pension_blog.pkl", "wb")
pickle.dump(pension_blog,f)
f.close()